In [1]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# from scipy.sparse import csr_matrix
# from scipy.sparse.linalg import svds
from surprise import SVDpp, Reader, Dataset
from surprise.model_selection import cross_validate

from typing import Union
from tqdm.notebook import tqdm

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

from src.metrics import reccomendation_report
from src.utils import surprise_predict
from src.utils import split_test_df
from src.utils import seed_everything
from src.utils import trainDataset
from src.utils import trainDatasetWithCrossFeatures

from src.models import wideAndDeep

import mmh3

## Constants

In [3]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [4]:
seed_everything(RANDOM_STATE)

## Data

In [5]:
df_movies = pd.read_csv(
    DATA_FOLDER + "movies.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["movieId", "name", "genre"],
)
df_ratings = pd.read_csv(
    DATA_FOLDER + "ratings.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["userId", "movieId", "rating", "timestamp"],
)
df_users = pd.read_csv(
    DATA_FOLDER + "users.csv",
    encoding="iso-8859-1",
    sep=";",
    names=["userId", "gender", "age", "occupation", "zip-code"],
)

In [6]:
## Encode usedId, movieId
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

df_movies["movieId"] = movie_encoder.fit_transform(df_movies["movieId"])
df_users["userId"] = user_encoder.fit_transform(df_users["userId"])

df_ratings["movieId"] = movie_encoder.transform(df_ratings["movieId"])
df_ratings["userId"] = user_encoder.transform(df_ratings["userId"])

#### Creating a test_user

In [7]:
df_users.loc[-1] = [df_users["userId"].max() + 1, "F", 21, 5, 777777]
df_users = df_users.reset_index(drop=True)
df_test_user = pd.DataFrame(columns=["userId", "movieId", "rating", "timestamp"])
df_test_user = df_test_user.assign(movieId=[6, 16, 3192, 1461, 827, 887, 593]).assign(
    userId=df_users["userId"].max(), rating=4, timestamp=lambda x: np.arange(x.shape[0])
)
print("Test user watch list:")
display(df_test_user.merge(df_movies, on="movieId"))
df_ratings = pd.concat([df_ratings, df_test_user], ignore_index=True)

Test user watch list:


,userId,movieId,rating,timestamp,name,genre
0,6040,6,4,0,Sabrina (1995),Comedy|Romance
1,6040,16,4,1,Sense and Sensibility (1995),Drama|Romance
2,6040,3192,4,2,Singles (1992),Comedy|Drama|Romance
3,6040,1461,4,3,Love and Other Catastrophes (1996),Romance
4,6040,827,4,4,Emma (1996),Comedy|Drama|Romance
5,6040,887,4,5,Singin' in the Rain (1952),Musical|Romance
6,6040,593,4,6,Pretty Woman (1990),Comedy|Romance


### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 10% of test.

In [8]:
df_ratings["rank"] = (
    df_ratings[["userId", "timestamp"]]
    .groupby("userId", as_index=False)["timestamp"]
    .rank(method="first", ascending=False)
)

In [9]:
# leave one out
df_train = df_ratings.loc[df_ratings["rank"] != 1].reset_index(drop=True)
df_test = (
    df_ratings.loc[df_ratings["rank"] == 1].reset_index(drop=True).assign(action=1)
)
df_test, df_val = train_test_split(df_test, test_size=0.2, random_state=RANDOM_STATE)

In [10]:
# enrich test data with 100 random movies from the ones not intercated by user
df_add = pd.DataFrame()
for user in tqdm(df_test.userId.unique(), desc="Enriching test"):
    movie = df_test.loc[df_test.userId == user, "movieId"]
    watched_movies = np.append(
        movie, df_train.loc[df_train.userId == user, "movieId"].values
    )
    not_wathed_movies = np.setdiff1d(
        np.arange(df_movies["movieId"].max() + 1), watched_movies
    )
    random_100 = np.random.choice(not_wathed_movies, 100, replace=False)

    df_temp = pd.DataFrame().assign(movieId=random_100, userId=user, action=0)
    df_add = pd.concat([df_add, df_temp], ignore_index=True)

df_test = pd.concat([df_test, df_add], ignore_index=True).drop(
    columns=["timestamp", "rating", "rank"]
)

df_add = pd.DataFrame()
for user in tqdm(df_val.userId.unique(), desc="Enriching val"):
    movie = df_val.loc[df_val.userId == user, "movieId"]
    watched_movies = np.append(
        movie, df_train.loc[df_train.userId == user, "movieId"].values
    )
    not_wathed_movies = np.setdiff1d(
        np.arange(df_movies["movieId"].max() + 1), watched_movies
    )
    random_100 = np.random.choice(not_wathed_movies, 100, replace=False)

    df_temp = pd.DataFrame().assign(movieId=random_100, userId=user, action=0)
    df_add = pd.concat([df_add, df_temp], ignore_index=True)

df_val = pd.concat([df_val, df_add], ignore_index=True).drop(
    columns=["timestamp", "rating", "rank"]
)

# adding data for test user
df_add = pd.DataFrame()
user = df_users["userId"].max()
movie = df_test.loc[df_test.userId == user, "movieId"]
watched_movies = np.append(
    movie, df_train.loc[df_train.userId == user, "movieId"].values
)
not_wathed_movies = np.setdiff1d(
    np.arange(df_movies["movieId"].max() + 1), watched_movies
)
random_500 = np.random.choice(not_wathed_movies, 500, replace=False)

df_temp = pd.DataFrame().assign(movieId=random_500, userId=user, action=0)
df_add = pd.concat([df_add, df_temp], ignore_index=True)
df_test_user = df_add

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1209 [00:00<?, ?it/s]

In [11]:
with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

## Building cross features

In [12]:
cross_feats_dim = 30

In [13]:
df_users["combined_feat"] = (
    df_users["gender"].astype(str)
    + df_users["age"].astype(str)
    + df_users["occupation"].astype(str)
)
user_features_mapping = {
    i: df_users.loc[df_users.userId == i, "combined_feat"].values[0]
    for i in df_users.userId.unique()
}
movie_features_mapping = {
    i: df_movies.loc[df_movies.movieId == i, "genre"].values[0]
    for i in df_movies.movieId.unique()
}

In [14]:
# cross features for test
test_idx = (
    (
        pd.Series(list(map(lambda x: user_features_mapping[x], df_test["userId"])))
        + pd.Series(list(map(lambda x: movie_features_mapping[x], df_test["movieId"])))
    )
    .apply(lambda x: mmh3.hash(x) % cross_feats_dim)
    .values
)
cross_test = torch.zeros(df_test.shape[0], cross_feats_dim)
cross_test[torch.arange(cross_test.shape[0]), test_idx] = 1

In [15]:
# cross features for test user
test_idx = (
    (
        pd.Series(list(map(lambda x: user_features_mapping[x], df_test_user["userId"])))
        + pd.Series(
            list(map(lambda x: movie_features_mapping[x], df_test_user["movieId"]))
        )
    )
    .apply(lambda x: mmh3.hash(x) % cross_feats_dim)
    .values
)
cross_test_user = torch.zeros(df_test_user.shape[0], cross_feats_dim)
cross_test_user[torch.arange(cross_test_user.shape[0]), test_idx] = 1

### WideAndDeep

In [16]:
seed_everything(RANDOM_STATE)
# cross features for train
train_loader = DataLoader(
    trainDatasetWithCrossFeatures(
        df_train,
        df_movies["movieId"].nunique(),
        user_features_mapping,
        movie_features_mapping,
        hash_bucket_size=cross_feats_dim,
    ),
    batch_size=2048,
    shuffle=True,
)

num_users = df_users["userId"].nunique()
num_items = df_movies["movieId"].nunique()

100%|█████████████████████████████| 4970875/4970875 [00:26<00:00, 186668.74it/s]


In [17]:
seed_everything(RANDOM_STATE)
wd = wideAndDeep(num_users, num_items, cross_feats_dim, mlp_layer_sizes=[16, 64, 32, 8])
display(wd)

wideAndDeep(
  (user_embedding): Embedding(6041, 8)
  (item_embedding): Embedding(3883, 8)
  (mlp): Sequential(
    (MLP_layer_1): Linear(in_features=16, out_features=64, bias=True)
    (Activation_layer_1): ReLU()
    (MLP_layer_2): Linear(in_features=64, out_features=32, bias=True)
    (Activation_layer_2): ReLU()
    (MLP_layer_3): Linear(in_features=32, out_features=8, bias=True)
    (Activation_layer_3): ReLU()
  )
  (cross): Linear(in_features=30, out_features=8, bias=True)
  (final): Linear(in_features=16, out_features=1, bias=True)
)

In [18]:
optimizer = torch.optim.Adam(wd.parameters(), lr=3e-4)
criterion = nn.BCELoss()
n_epochs = 5
device = "cpu"

In [19]:
### Train Wide and Deep
wd.to(device)
num_iterations = len(train_loader)

for epoch in tqdm(range(n_epochs), desc="Epochs"):
    # train
    total_train_loss = 0
    wd.train()
    with tqdm(train_loader, unit="batch") as tepoch:
        for userIds, movieIds, cross, ratings in tepoch:
            pred_train = wd(userIds.to(device), movieIds.to(device), cross.to(device))
            loss_train = criterion(pred_train.flatten(), ratings.to(device))

            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
            total_train_loss += loss_train.item()
            tepoch.set_postfix(loss=loss_train.item())
    print("Epoch:", epoch)
    print("Train loss", round(total_train_loss / num_iterations, 5))

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 0
Train loss 0.45849


  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 1
Train loss 0.36831


...

Epoch: 2
Train loss 0.35566


  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 3
Train loss 0.35308


  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 4
Train loss 0.35204


In [20]:
test_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test["userId"]), torch.tensor(df_test["movieId"]), cross_test
    ),
    batch_size=4096,
    shuffle=False,
)

In [21]:
# predict wd
wd.eval()
total_preds = torch.zeros(len(test_loader.dataset))
batch_size = test_loader.batch_size
for i, (userIds, movieIds, cross) in enumerate(
    tqdm(test_loader, desc="Inference", unit="batch")
):
    with torch.no_grad():
        total_preds[i * batch_size : (i + 1) * batch_size] = wd(
            userIds, movieIds, cross
        ).flatten()

Inference:   0%|          | 0/120 [00:00<?, ?batch/s]

In [22]:
df_test["rating_pred"] = total_preds.numpy()
pred, target, pred_items = split_test_df(
    df_test, "userId", "movieId", "rating_pred", "action"
)
reccomendation_report(pred, target, pred_items, cos_dist, popularity, k=15)

{'Hit rate @ K': tensor(0.5611),
 'NDCG @ K': tensor(0.2840),
 'Diversity (ILD)': tensor(0.1458),
 'Novelty (EPC)': tensor(0.6848)}

## Test user recommendations

In [23]:
pd.set_option("display.max_rows", 102)

In [24]:
test_user_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test_user["userId"]),
        torch.tensor(df_test_user["movieId"]),
        cross_test_user,
    ),
    batch_size=4096,
    shuffle=False,
)

# predict wd
wd.eval()
total_preds = torch.zeros(len(test_user_loader.dataset))
batch_size = test_user_loader.batch_size
for i, (userIds, movieIds, cross) in enumerate(
    tqdm(test_user_loader, desc="Inference", unit="batch")
):
    with torch.no_grad():
        total_preds[i * batch_size : (i + 1) * batch_size] = wd(
            userIds, movieIds, cross
        ).flatten()

df_test_user["rating_pred"] = total_preds.numpy()

display(
    df_test_user.loc[df_test_user.userId == df_test_user["userId"].max()]
    .sort_values(by="rating_pred", ascending=False)
    .merge(df_movies, on="movieId")
    .loc[:100, ["userId", "movieId", "name", "genre"]]
)

Inference:   0%|          | 0/1 [00:00<?, ?batch/s]

,userId,movieId,name,genre
0,6040,315,"Shawshank Redemption, The (1994)",Drama
1,6040,1575,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
2,6040,847,"Godfather, The (1972)",Action|Crime|Drama
3,6040,2847,Total Recall (1990),Action|Adventure|Sci-Fi|Thriller
4,6040,1366,Jaws (1975),Action|Horror
5,6040,49,"Usual Suspects, The (1995)",Crime|Thriller
6,6040,2728,Big (1988),Comedy|Fantasy
7,6040,724,"Rock, The (1996)",Action|Adventure|Thriller
8,6040,3412,High Fidelity (2000),Comedy
9,6040,1372,Jerry Maguire (1996),Drama|Romance
